In [1]:
'''
This notebook is concerned with the actual modelling and training
of the dataset to produce accurate predictions
'''

'\nThis notebook is concerned with the actual modelling and training\nof the dataset to produce accurate predictions\n'

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
# from sklearn import li
import sklearn as sk
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, Lasso
from sklearn.model_selection import cross_validate, cross_val_score, KFold
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
# import lightgbm as lgb

In [4]:
df_train = pd.read_csv("C:/Users/Asus/jypNotebooks/housing predictions/data/train.csv", engine='python')
df_test = pd.read_csv("C:/Users/Asus/jypNotebooks/housing predictions/data/test.csv", engine='python')

In [5]:
# total = df_train.isnull().sum().sort_values(ascending=False)
# percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# # missing_data.head(20)

In [6]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
# #dealing with the missing data
# df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
# df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
# #checking there is no missing data
# df_train.isnull().sum().max()



In [8]:
'''
Concatinating the test and train set so as to perform feature engineering
on them both as a whole
feature engineering in this case involves solving missing values, normalization,
creating other features and solving outliers among others
'''

'\nConcatinating the test and train set so as to perform feature engineering\non them both as a whole\nfeature engineering in this case involves solving missing values, normalization,\ncreating other features and solving outliers among others\n'

In [9]:
all_data = pd.concat((df_train.loc[:,'MSSubClass':'SaleCondition'],
                      df_test.loc[:,'MSSubClass':'SaleCondition']))

In [10]:
all_data.shape

(2919, 79)

In [11]:
all_data.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [12]:
'''
we are going to log transform numerical values to solve skweness and normalize the data

'''

'\nwe are going to log transform numerical values to solve skweness and normalize the data\n\n'

In [13]:
#log transform the target
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])
#idenitying numeric features 
numeric_feats = all_data.dtypes[all_data.dtypes != 'object'].index
#computing skewness
skewed_feats = df_train[numeric_feats].apply(lambda x: x.skew())
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])#using log to remove skewness

# numeric_values = all_data[numeric_feats]

#identifying and removing outliers in the numerical features


In [14]:
'''
Categorical values are values that are not numerical eg good, bad, excellent
we need to transform this into a language that the computer can understand
There are several ways of doing this eg one-hot encoding
here we will use a pandas fun pd.get_dummies()
'''

'\nCategorical values are values that are not numerical eg good, bad, excellent\nwe need to transform this into a language that the computer can understand\nThere are several ways of doing this eg one-hot encoding\nhere we will use a pandas fun pd.get_dummies()\n'

In [15]:
all_data = pd.get_dummies(all_data) #replacing categorical values
print(all_data.shape)

(2919, 288)


In [16]:
all_data.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=288)

In [17]:
'''
Here I chose to handle missing values by filling them with the mean of the column
There are other several ways to solve missing values - check this out
'''

'\nHere I chose to handle missing values by filling them with the mean of the column\nThere are other several ways to solve missing values - check this out\n'

In [18]:
#filling NA's with the mean of the column
all_data = all_data.fillna(all_data.mean()) 


In [19]:
all_data.isnull().sum().max()

0

In [20]:
print(all_data.shape)

(2919, 288)


In [21]:
# #normalizing saleprice
# saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis])

In [22]:
#log transforming saleprice instead to see if it will improve score
# df_train['SalePrice'] = np.log1p(df_train["SalePrice"])
# sp =  np.log1p(df_train["SalePrice"])
#side note -- this has tremendously improved the score from 0.31 to 0.0012

In [23]:
df_train['SalePrice'].values

array([12.24769912, 12.10901644, 12.31717117, ..., 12.49313327,
       11.86446927, 11.90159023])

In [24]:
# sptype= isinstance(sp, pd.Series)
# print(sptype)

In [25]:
'''
Here I am splitting all_data into the test and train set.
  
'''

'\nHere I am splitting all_data into the test and train set.\n  \n'

In [26]:
X_train = all_data[:df_train.shape[0]]
X_test = all_data[df_train.shape[0]:]
y = df_train['SalePrice']

In [27]:
print(X_train.shape)
print(X_test.shape)

(1460, 288)
(1459, 288)


In [28]:
print(X_train.shape)

(1460, 288)


In [29]:
'''
Before we begin training we must define a cross-validation strategy
This how we are going to evaluate our model
n_folds represents the number of times we are going to split our training set for evaluation
This helps with shuffling the dataset and making the model more generalized
Here I am using the KFold strategy for cross-validation. There are many other strategies -- check this out
'''

'\nBefore we begin training we must define a cross-validation strategy\nThis how we are going to evaluate our model\nn_folds represents the number of times we are going to split our training set for evaluation\nThis helps with shuffling the dataset and making the model more generalized\nHere I am using the KFold strategy for cross-validation. There are many other strategies -- check this out\n'

In [30]:
#defining a cross validation strategy
n_folds = 5
def rmse_cv(model):
#adding code to shuffle the dataset -- side note -it hasnt really affected the score
    kf = KFold(n_folds, shuffle =True, random_state =42).get_n_splits(all_data.values)
    rmse=np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv=kf))
#     rms=abs(rmse)
    return(rmse)


In [31]:
'''
Now we begin modeling. There are several regression models that you can use
I am using lasso and elastic net regression in this one
others inclued ridge regression
The make_pipeline() class from sklearn helps you to define the steps during training(pipeline)
So in the pipline I am first using RobustScaler() to make my model robust to outliers.
Outliers are extreem observations in the dataset that can make the model perform poorly
'''

'\nNow we begin modeling. There are several regression models that you can use\nI am using lasso and elastic net regression in this one\nothers inclued ridge regression\nThe make_pipeline() class from sklearn helps you to define the steps during training(pipeline)\nSo in the pipline I am first using RobustScaler() to make my model robust to outliers.\nOutliers are extreem observations in the dataset that can make the model perform poorly\n'

In [32]:
# using lasso regression
#RobustScaler is used to make the model more robust ie solve outliers
lasso = make_pipeline(RobustScaler(), LassoCV(alphas = [1, 0.1, 0.001, 0.0005]))
lasso.fit(X_train, y)                     


Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('lassocv',
                 LassoCV(alphas=[1, 0.1, 0.001, 0.0005], copy_X=True, cv=None,
                         eps=0.001, fit_intercept=True, max_iter=1000,
                         n_alphas=100, n_jobs=None, normalize=False,
                         positive=False, precompute='auto', random_state=None,
                         selection='cyclic', tol=0.0001, verbose=False))],
         verbose=False)

In [33]:
'''
Here I am making predictions using the lasso model
np.expm1 reverts the scale back to the way it was. Rember we log transformed 
numerical values to solve skeweness
'''

'\nHere I am making predictions using the lasso model\nnp.expm1 reverts the scale back to the way it was. Rember we log transformed \nnumerical values to solve skeweness\n'

In [34]:
lasso_preds = np.expm1(lasso.predict(X_test))

In [35]:
'''This is now the mean cross-valiadation value on all the five folds'''

'This is now the mean cross-valiadation value on all the five folds'

In [36]:
rmse_cv(lasso).mean()

0.12254298849240777

In [37]:
#using Elastic net regression
Enet = make_pipeline(RobustScaler(), ElasticNet(alpha = 0.0005, l1_ratio=.7))
Enet.fit(X_train, y)

Pipeline(memory=None,
         steps=[('robustscaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('elasticnet',
                 ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True,
                            l1_ratio=0.7, max_iter=1000, normalize=False,
                            positive=False, precompute=False, random_state=None,
                            selection='cyclic', tol=0.0001,
                            warm_start=False))],
         verbose=False)

In [38]:
Enet_preds = np.expm1(Enet.predict(X_test))

In [39]:
rmse_cv(Enet).mean()

0.12307709235108763

In [40]:
'''
This commented out part was experimenting with gradient boosting algs - check this out especially xgboost 
in an attempt to improve the score
I also tried stacking the models together for a better score.Still a work in progeress
'''

'\nThis commented out part was experimenting with gradient boosting algs - check this out especially xgboost \nin an attempt to improve the score\nI also tried stacking the models together for a better score.Still a work in progeress\n'

In [41]:
# def rmsexg(y , y_pred):
#     return np.sqrt(mean_squared_error(y, y_pred))

In [42]:
# #xgb boost
# xgbr = xgb.XGBRegressor()
# xgbmodel = xgbr.fit(X_train, y)
# xgb_train_pred = xgbmodel.predict(X_train)
# # print(xgb_pred)
# print(rmsexg(y, xgb_train_pred))


In [43]:
# xgb_pred = np.exp(xgbmodel.predict(X_test))
# # print(rmsexg(y, xgb_pred))

In [44]:
# sorted(sk.metrics.SCORERS.keys())

In [45]:
# #creating a class to stack all the predictors
# class Avrgmodels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models = models
        
#     #defining clones of the original models to fit the data in
#     def fit(self, X, y):
#         self.models_ = [clone(x) for x in self.models]
#         #training cloned models
#         for model in self.models_:
#             model.fit(X, y)
            
#         return self
#     def predict(self, X):
#         preds = np.column_stack([model.predict(X) for model in self.models_])
#         return np.mean(preds, axis=1)


In [46]:
# avrged_models = Avrgmodels(models = (lasso, Enet, xgbr))
# score = rmse_cv(avrged_models).mean()
# print(score)

In [47]:
# avrged_models.fit(X_train, y)
# stacked_pred = avrged_models.predict(X_train)
# # print(stacked_pred)

In [48]:
'''
Here I am doing the actual predictions. The two models have an equal 
effect on my predictions
'''

'\nHere I am doing the actual predictions. The two models have an equal \neffect on my predictions\n'

In [49]:
#choosing xgb because it has the highest score
predictions =   0.5*lasso_preds + 0.5*Enet_preds

In [50]:
'''
Solution to csv file
'''

'\nSolution to csv file\n'

In [51]:
solution = pd.DataFrame({"id":df_test.Id, "SalePrice":predictions})
solution.to_csv("submissions.csv", index = False)

In [ ]:
'''
Things to focus on
visualisations and insights from visualizations
solving missing values
noticing and solving outliers
normalizing features for training (log transformation and one-hot encoding)
how you define the cross-val strategy
the regression models used and why regression(it is a linear problem with one dependent variable)
how you hv improved the model performance - gradient boosting and stacking models
how you hv reduced bias(underfitting) and variance(overfitting) - check out regularization
Here , I used trial and error to get the best regularization value.
Discuss the datasets you have training set, test set and public test set found on kaggle

'''